In [1]:
from marubatsu import Marubatsu
from ai import ai_abs_tt4, ai2s, ai14s

mb = Marubatsu()

In [2]:
%%timeit
ai_abs_tt4(mb)

310 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit
ai2s(mb)

345 µs ± 753 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%%timeit
ai14s(mb)

519 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
from ai import ai_by_score, dprint
from copy import deepcopy

@ai_by_score
def ai_abs_all(mb, debug=False, shortest_victory=False,
               init_ab=False, use_tt=False, ai_for_mo=None,
               params={}, sort_allnodes=False, calc_count=False):        
    count = 0
    def ab_search(mborig, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        count += 1
        if mborig.status == Marubatsu.CIRCLE:
            return (11 - mborig.move_count) / 2 if shortest_victory else 1
        elif mborig.status == Marubatsu.CROSS:
            return (mborig.move_count - 10) / 2 if shortest_victory else -1
        elif mborig.status == Marubatsu.DRAW:
            return 0
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if ai_for_mo is not None:
            bestmove = ai_for_mo(mborig, **params)
            index = legal_moves.index(bestmove)
            legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]            
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = max(score, ab_search(mb, tt, alpha, beta))
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = min(score, ab_search(mb, tt, alpha, beta))
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        if use_tt:
            from util import calc_same_boardtexts

            boardtxtlist = calc_same_boardtexts(mborig)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt in boardtxtlist:
                tt[boardtxt] = (lower_bound, upper_bound)
        return score
                
    min_score = -2 if shortest_victory else -1
    max_score = 3 if shortest_victory else 1
    alpha = min_score if init_ab else float("-inf")
    beta = max_score if init_ab else float("inf")

    score = ab_search(mb, {}, alpha=alpha, beta=beta)
    dprint(debug, "count =", count)
    if calc_count:
        return count
    if mb.turn == Marubatsu.CIRCLE:
        score *= -1
    return score

In [6]:
from util import Check_solved

for ai_for_mo in [None, ai2s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, use_tt: {use_tt}, init_ab: {init_ab}")
                params = {
                    "ai_for_mo": ai_for_mo,
                    "shortest_victory": shortest_victory,
                    "init_ab": init_ab,
                    "use_tt": use_tt,
                }
                Check_solved.is_strongly_solved(ai_abs_all, params=params)

ai_for_mo: None, sv: False, use_tt: False, init_ab: False


  0%|          | 0/431 [00:00<?, ?it/s]

100%|██████████| 431/431 [00:03<00:00, 108.73it/s]


431/431 100.00%
ai_for_mo: None, sv: False, use_tt: False, init_ab: True


100%|██████████| 431/431 [00:03<00:00, 140.83it/s]


...

431/431 100.00%
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: True, use_tt: True, init_ab: False


100%|██████████| 431/431 [00:05<00:00, 82.08it/s] 


431/431 100.00%
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: True, use_tt: True, init_ab: True


100%|██████████| 431/431 [00:04<00:00, 86.67it/s] 

431/431 100.00%


In [7]:
from ai import ai10s

for ai_for_mo in [None, ai2s, ai10s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, use_tt: {use_tt}, init_ab: {init_ab}")
                print("count=", ai_abs_all(mb, ai_for_mo=ai_for_mo,
                                          shortest_victory=shortest_victory,
                                          init_ab=init_ab, use_tt=use_tt,
                                          calc_score=True, calc_count=True))

ai_for_mo: None, sv: False, use_tt: False, init_ab: False
count= 18297
ai_for_mo: None, sv: False, use_tt: False, init_ab: True
count= 16811
ai_for_mo: None, sv: False, use_tt: True, init_ab: False
count= 1054
ai_for_mo: None, sv: False, use_tt: True, init_ab: True
count= 832
ai_for_mo: None, sv: True, use_tt: False, init_ab: False
count= 20866
ai_for_mo: None, sv: True, use_tt: False, init_ab: True
count= 20484
ai_for_mo: None, sv: True, use_tt: True, init_ab: False
count= 1275
ai_for_mo: None, sv: True, use_tt: True, init_ab: True
count= 1242
ai_for_mo: <function ai2s at 0x00000216EC38FCE0>, sv: False, use_tt: False, init_ab: False
count= 14810
ai_for_mo: <function ai2s at 0x00000216EC38FCE0>, sv: False, use_tt: False, init_ab: True
count= 11274
ai_for_mo: <function ai2s at 0x00000216EC38FCE0>, sv: False, use_tt: True, init_ab: False
count= 884
ai_for_mo: <function ai2s at 0x00000216EC38FCE0>, sv: False, use_tt: True, init_ab: True
count= 734
ai_for_mo: <function ai2s at 0x00000216EC

In [8]:
from pprint import pprint

result = ai14s(mb, analyze=True)
pprint(result)

{'candidate': [(1, 1)],
 'score_by_move': {(0, 0): 1.5,
                   (0, 1): 1.0,
                   (0, 2): 1.5,
                   (1, 0): 1.0,
                   (1, 1): 2.0,
                   (1, 2): 1.0,
                   (2, 0): 1.5,
                   (2, 1): 1.0,
                   (2, 2): 1.5}}


In [9]:
score_by_move = result["score_by_move"]
pprint(score_by_move)

{(0, 0): 1.5,
 (0, 1): 1.0,
 (0, 2): 1.5,
 (1, 0): 1.0,
 (1, 1): 2.0,
 (1, 2): 1.0,
 (2, 0): 1.5,
 (2, 1): 1.0,
 (2, 2): 1.5}


In [31]:
pprint(sorted(score_by_move))

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


In [ ]:
for item in score_by_move.items():
    print(item)

((0, 0), 1.5)
((1, 0), 1.0)
((2, 0), 1.5)
((0, 1), 1.0)
((1, 1), 2.0)
((2, 1), 1.0)
((0, 2), 1.5)
((1, 2), 1.0)
((2, 2), 1.5)


In [12]:
items = score_by_move.items()
items[0]

TypeError: 'dict_items' object is not subscriptable

In [13]:
score_by_move_list = list(items)
pprint(score_by_move_list)

[((0, 0), 1.5),
 ((1, 0), 1.0),
 ((2, 0), 1.5),
 ((0, 1), 1.0),
 ((1, 1), 2.0),
 ((2, 1), 1.0),
 ((0, 2), 1.5),
 ((1, 2), 1.0),
 ((2, 2), 1.5)]


In [14]:
score_by_move_list.sort(key=lambda x:x[1])
pprint(score_by_move_list)

[((1, 0), 1.0),
 ((0, 1), 1.0),
 ((2, 1), 1.0),
 ((1, 2), 1.0),
 ((0, 0), 1.5),
 ((2, 0), 1.5),
 ((0, 2), 1.5),
 ((2, 2), 1.5),
 ((1, 1), 2.0)]


In [15]:
score_by_move_list = sorted(items, key=lambda x:x[1])
pprint(score_by_move_list)

[((1, 0), 1.0),
 ((0, 1), 1.0),
 ((2, 1), 1.0),
 ((1, 2), 1.0),
 ((0, 0), 1.5),
 ((2, 0), 1.5),
 ((0, 2), 1.5),
 ((2, 2), 1.5),
 ((1, 1), 2.0)]


In [16]:
score_by_move_list = sorted(score_by_move.items(), key=lambda x:x[1])
pprint(score_by_move_list)

[((1, 0), 1.0),
 ((0, 1), 1.0),
 ((2, 1), 1.0),
 ((1, 2), 1.0),
 ((0, 0), 1.5),
 ((2, 0), 1.5),
 ((0, 2), 1.5),
 ((2, 2), 1.5),
 ((1, 1), 2.0)]


In [17]:
score_by_move_list.sort(key=lambda x:x[0])
pprint(score_by_move_list)

[((0, 0), 1.5),
 ((0, 1), 1.0),
 ((0, 2), 1.5),
 ((1, 0), 1.0),
 ((1, 1), 2.0),
 ((1, 2), 1.0),
 ((2, 0), 1.5),
 ((2, 1), 1.0),
 ((2, 2), 1.5)]


In [18]:
pprint(sorted(score_by_move.items(), reverse=True))

[((2, 2), 1.5),
 ((2, 1), 1.0),
 ((2, 0), 1.5),
 ((1, 2), 1.0),
 ((1, 1), 2.0),
 ((1, 0), 1.0),
 ((0, 2), 1.5),
 ((0, 1), 1.0),
 ((0, 0), 1.5)]


In [19]:
score_by_move_list.sort(key=lambda x:x[1], reverse=True)
pprint(score_by_move_list)

[((1, 1), 2.0),
 ((0, 0), 1.5),
 ((0, 2), 1.5),
 ((2, 0), 1.5),
 ((2, 2), 1.5),
 ((0, 1), 1.0),
 ((1, 0), 1.0),
 ((1, 2), 1.0),
 ((2, 1), 1.0)]


In [20]:
ordered_score_by_move = dict(score_by_move_list)
pprint(ordered_score_by_move, sort_dicts=False)

{(1, 1): 2.0,
 (0, 0): 1.5,
 (0, 2): 1.5,
 (2, 0): 1.5,
 (2, 2): 1.5,
 (0, 1): 1.0,
 (1, 0): 1.0,
 (1, 2): 1.0,
 (2, 1): 1.0}


In [21]:
pprint(ordered_score_by_move)

{(0, 0): 1.5,
 (0, 1): 1.0,
 (0, 2): 1.5,
 (1, 0): 1.0,
 (1, 1): 2.0,
 (1, 2): 1.0,
 (2, 0): 1.5,
 (2, 1): 1.0,
 (2, 2): 1.5}


In [22]:
ordered_move_list = [x[0] for x in score_by_move_list]
print(ordered_move_list)

[(1, 1), (0, 0), (0, 2), (2, 0), (2, 2), (0, 1), (1, 0), (1, 2), (2, 1)]


In [23]:
ordered_move_list = list(ordered_score_by_move.keys())
print(ordered_move_list)

[(1, 1), (0, 0), (0, 2), (2, 0), (2, 2), (0, 1), (1, 0), (1, 2), (2, 1)]


In [24]:
result = ai14s(mb, analyze=True)
score_by_move = result["score_by_move"]
items = score_by_move.items()
score_by_move_list = list(items)
score_by_move_list.sort(key=lambda x:x[1], reverse=True)
orderd_move_list = [x[0] for x in score_by_move_list]
print(orderd_move_list)

[(1, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0), (0, 1), (2, 1), (1, 2)]


In [25]:
score_by_move = ai14s(mb, analyze=True)["score_by_move"]
score_by_move_list = sorted(score_by_move.items(), key=lambda x:x[1], reverse=True)
orderd_move_list = [x[0] for x in score_by_move_list]
print(orderd_move_list)

[(1, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0), (0, 1), (2, 1), (1, 2)]


In [26]:
orderd_move_list = [x[0] for x in sorted(ai14s(mb, analyze=True)["score_by_move"].items(), key=lambda x:x[1], reverse=True)]
print(orderd_move_list)

[(1, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0), (0, 1), (2, 1), (1, 2)]


In [28]:
for ai_for_mo in [None, ai2s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, use_tt: {use_tt}, init_ab: {init_ab}")
                params = {
                    "ai_for_mo": ai_for_mo,
                    "shortest_victory": shortest_victory,
                    "init_ab": init_ab,
                    "use_tt": use_tt,
                    "sort_allnodes": True,
                }
                Check_solved.is_strongly_solved(ai_abs_all, params=params)

ai_for_mo: None, sv: False, use_tt: False, init_ab: False


100%|██████████| 431/431 [00:03<00:00, 110.35it/s]


431/431 100.00%
ai_for_mo: None, sv: False, use_tt: False, init_ab: True


100%|██████████| 431/431 [00:03<00:00, 137.48it/s]


431/431 100.00%
ai_for_mo: None, sv: False, use_tt: True, init_ab: False


...

431/431 100.00%
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: True, use_tt: True, init_ab: False


100%|██████████| 431/431 [00:05<00:00, 79.54it/s] 


431/431 100.00%
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: True, use_tt: True, init_ab: True


100%|██████████| 431/431 [00:05<00:00, 85.30it/s] 

431/431 100.00%


In [29]:
for ai_for_mo in [ai10s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, init_ab: {init_ab}, use_tt: {use_tt}")
                print("count=", ai_abs_all(mb, ai_for_mo=ai_for_mo,
                                          shortest_victory=shortest_victory,
                                          init_ab=init_ab, use_tt=use_tt,
                                          sort_allnodes=True, calc_score=True, calc_count=True))

ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: False, use_tt: False
count= 2305
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: True, use_tt: False
count= 2082
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: False, use_tt: True
count= 285
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: True, use_tt: True
count= 260
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: False, use_tt: False
count= 2305
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: True, use_tt: False
count= 2305
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: False, use_tt: True
count= 285
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: True, use_tt: True
count= 285
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: False, init_ab: False, use_tt: False
count= 1496
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: False, init_ab: True, use

In [30]:
from util import load_mblist

mblist = load_mblist("../data/mblist_by_board_min.dat")
for ai_for_mo in [ai10s, ai14s]:
    for shortest_victory in [False, True]:
        for use_tt in [False, True]:
            for init_ab in [False, True]:
                print(f"ai_for_mo: {ai_for_mo}, sv: {shortest_victory}, init_ab: {init_ab}, use_tt: {use_tt}")
                count = 0
                count2 = 0
                for mb in mblist:
                    count += ai_abs_all(mb, ai_for_mo=ai_for_mo,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        sort_allnodes=False, calc_score=True, calc_count=True)
                    count2 += ai_abs_all(mb, ai_for_mo=ai_for_mo,
                                        shortest_victory=shortest_victory,
                                        init_ab=init_ab, use_tt=use_tt,
                                        sort_allnodes=True, calc_score=True, calc_count=True)
                print("count=", count)
                print("count2=", count2)

ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: False, use_tt: False
count= 13816
count2= 13214
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: True, use_tt: False
count= 8745
count2= 7938
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: False, use_tt: True
count= 10371
count2= 9904
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: False, init_ab: True, use_tt: True
count= 6625
count2= 5976
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: False, use_tt: False
count= 15048
count2= 13950
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: True, use_tt: False
count= 13131
count2= 11818
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: False, use_tt: True
count= 11082
count2= 10300
ai_for_mo: <function ai10s at 0x00000216EC394D60>, sv: True, init_ab: True, use_tt: True
count= 9806
count2= 8811
ai_for_mo: <function ai14s at 0x00000216EC395260>, sv: False, init_